<a href="https://colab.research.google.com/github/pedroarthurob/NLP-FinalProject/blob/main/Projeto_Final_PLN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando todas as dependências do Projeto.

### Downloads

In [1]:
!pip install 'crewai[tools]'

In [2]:
!pip install gdown

### Importações essenciais

In [3]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

import os
from google.colab import userdata

from crewai_tools import PDFSearchTool
from crewai import Agent, Crew, Process, Task

import gdown

from IPython.display import Markdown

## Configuração da chave da API

In [4]:
OPENAI_API_KEY = userdata.get('OPEN_AI_KEY')
model_ID = userdata.get('GPT_MODEL')


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["OPENAI_MODEL_NAME"] = model_ID

## Download das Provas e Gabaritos do ENEM (2011-2023)

In [5]:
!gdown --id 1ahPqSEA6vaqxP8tzo2lQmiuQdItOU6aI

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ahPqSEA6vaqxP8tzo2lQmiuQdItOU6aI
From (redirected): https://drive.google.com/uc?id=1ahPqSEA6vaqxP8tzo2lQmiuQdItOU6aI&confirm=t&uuid=744f7b2f-0f1f-446b-9ddf-3d1b5e8434e3
To: /content/provas_gabaritos_combinados.pdf
100% 144M/144M [00:02<00:00, 63.1MB/s]


## Download notas ENEM

In [6]:
!gdown --id 1-W37X73MtYKxhOvXKILGwIaof5TH7QFC

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1-W37X73MtYKxhOvXKILGwIaof5TH7QFC
To: /content/TRI Enem.pdf
100% 54.2k/54.2k [00:00<00:00, 62.9MB/s]


## Atribuição do arquivo ao pdf_tool(CrewAI Tool)

In [7]:
pdf_tool_provas = PDFSearchTool(directory='/content/provas_gabaritos_combinados.pdf')

In [8]:
pdf_tool_notas = PDFSearchTool(directory='/content/TRI Enem.pdf')

## Criação dos Agentes

In [44]:
agente_diagnostico = Agent(
    role= 'Diagnosticar o candidato',
    goal= 'Receber as notas de linguagens {linguagens}, humanas {humanas}, natureza {natureza}, matemática {matemática} e redação {redação} do candidato,'
          'e comparar com as notas mínimas e máximas possíveis apresentadas no PDF de notas do ENEM, avaliando assim o desempenho do candidato',
    backstory= 'Especialista em analisar candidatos do ENEM com base no seu desempenho em ',
    verbose=True,
    memory=True,
    tools=[pdf_tool_notas]
)

In [45]:
agente_gerador_de_planos = Agent(
    role= 'Gerador de Planos de estudos',
    goal= 'Criar um plano de estudos personalizado para o usuário com base no diagnóstico do desempenho do mesmo',
    backstory= 'Especialista em planejamento educacional e otimização de sessões de estudo para exames, especializado no ENEM',
    verbose=True,
    memory=True
)

In [46]:
agente_estrategista = Agent(
    role= 'Estrategista de estudos e resolução de provas do ENEM',
    goal= 'Sugerir estratégias de estudo e resolução de provas do ENEM com base no diagnóstico do desempenho do mesmo',
    backstory= 'Especialista em diferentes estratégias de estudo e resolução de provas do ENEM',
    verbose=True,
    memory=True
)

In [47]:
agente_pesquisador_topico = Agent(
    role='Pesquisador de Tópicos',
    goal='Buscar informações relevantes sobre {tópico} nos arquivos PDF do ENEM fornecidos.',
    verbose=True,
    memory=True,
    backstory='Especialista em analisar o conteúdo das provas e identificar os tópicos principais com base nos exames anteriores do ENEM.',
    tools=[pdf_tool_provas],
    allow_delegation=True
)


In [48]:
agente_sumarizador_topico = Agent(
    role='Produzir resumos de tópicos com foco no ENEM',
    goal='Escrever resumos que auxiliem os estudantes a compreender tópicos e conceitos para o ENEM',
    verbose=True,
    memory=True,
    backstory='Especialista em produzir resumos de conteúdos para candidatos ao ENEM',
    allow_delegation=True
)


## Criação das Tasks

In [49]:
tarefa_diagnostico = Task(
    description='Realizar uma análise detalhada do desempenho do candidato com base nas notas de '
                'linguagens {linguagens}, humanas {humanas}, natureza {natureza}, matemática {matemática} e redação {redação} do candidato,'
                'comparando-as com as médias e distribuições de notas das edições anteriores do ENEM extraídas dos arquivos PDF.',
    expected_output='Um relatório detalhado em formato markdown que inclua: (1) análise comparativa de desempenho em cada área do conhecimento,'
                    '(2) identificação de pontos fortes e fracos com base nas notas comparadas às médias históricas,'
                    '(3) sugestões específicas de melhoria em cada área com base nos resultados, (4) uma visão geral do progresso potencial do candidato.',
    agent=agente_diagnostico,
)


In [50]:
tarefa_plano_estudos = Task(
    description='Gerar um plano de estudos personalizado com base no diagnóstico de desempenho do candidato,'
                'focado em melhorar suas áreas de dificuldade e manter o desempenho nas áreas fortes.',
    expected_output="Um plano de estudos em formato markdown, organizado em tópicos diários,"
                    'incluindo sugestões de questões para praticar e recursos adicionais'
                    '(como vídeos ou leituras recomendadas), com recomendações de tempo de estudo para cada área do conhecimento.',
    agent=agente_gerador_de_planos,
    context=[tarefa_diagnostico],
    output_file='/content/plano_de_estudos.md'
)


In [51]:
tarefa_estrategias = Task(
    description='Propor estratégias de estudo e resolução de prova personalizadas com base no diagnóstico de desempenho do candidato.',
    expected_output='Um relatório em formato markdown que inclua: (1) uma análise dos principais erros e acertos do candidato,'
                    '(2) um plano de estudo personalizado com estratégias de priorização de tópicos e tempo,'
                    '(3) dicas práticas de resolução de questões com foco nas áreas de maior dificuldade, e'
                    '(4) sugestões de técnicas de revisão e gerenciamento de tempo para melhorar o desempenho geral nas provas.',
    agent=agente_estrategista,
    context=[tarefa_diagnostico],
    output_file='/content/estrategias.md'
)


In [52]:
tarefa_buscar_topico = Task(
    description='Pesquisar informações detalhadas sobre o {tópico} nos arquivos PDF das provas do ENEM (2011-2023),'
                'incluindo uma análise das questões relacionadas.',
    expected_output='Um relatório em formato markdown que inclua: (1) uma visão geral do tópico pesquisado,'
                    '(2) as questões mais frequentes relacionadas ao tópico, (3) como o tópico foi abordado ao longo dos anos,'
                    '(4) recomendações de foco para o estudo, com base nas áreas mais recorrentes ou desafiadoras do exame.',
    agent=agente_pesquisador_topico
)


In [53]:
tarefa_produzir_resumo = Task(
    description='Gerar resumos claros e concisos para auxiliar os estudantes na compreensão do {tópico} abordado no ENEM,'
                'com base nas informações detalhadas do relatório sobre o {tópico}.',
    expected_output='Um resumo em formato markdown que explique de forma simplificada os principais conceitos do tópico,'
                    ' com foco nos pontos mais cobrados no ENEM, exemplos relevantes e sugestões de como abordar o tema durante a prova.',
    agent=agente_sumarizador_topico,
    context=[tarefa_buscar_topico],
    output_file='/content/resumo.md'
)


## Criação das Crews

In [54]:
plano_de_estudos_crew = Crew (
    agents=[agente_diagnostico, agente_gerador_de_planos],
    tasks=[tarefa_diagnostico, tarefa_plano_estudos],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True
)

In [55]:
estrategias_crew = Crew (
    agents=[agente_diagnostico, agente_estrategista],
    tasks=[tarefa_diagnostico, tarefa_estrategias],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True
)

In [56]:
resumo_crew = Crew (
    agents=[agente_pesquisador_topico, agente_sumarizador_topico],
    tasks=[tarefa_buscar_topico, tarefa_produzir_resumo],
    verbose=True,
    process=Process.sequential,
    memory=True,
    cache=True,
    share_crew=True
)

## Funções para lidar com as entradas do usuário e dar o kickoff da crew correta

In [57]:
# Função para criar o plano de estudos
def criar_plano_de_estudos():
    while True:
        notas = input("Informe suas últimas notas obtidas em linguagens, humanas, natureza, matemática e redação\n(utlize '.' para as notas e as separe por vírgula): ").split(',')
        notas = [t.strip() for t in notas]  # Remover espaços em branco

        # Verificar se a quantidade de notas é a esperada
        if len(notas) == 5 and all(nota.replace('.', '', 1).isdigit() for nota in notas):
            inputs = {
                'linguagens': notas[0],
                'humanas': notas[1],
                'natureza': notas[2],
                'matemática': notas[3],
                'redação': notas[4]
            }
            results = plano_de_estudos_crew.kickoff(inputs=inputs)
            print("Plano de estudos criado com sucesso!")
            return results
        else:
            print("Entrada inválida. Por favor, insira 5 notas válidas separadas por vírgula.")

def pesquisar_topico():
    while True:
        topicos_input = input("Por favor, informe os tópicos que você gostaria de estudar, separados por vírgula: ")
        topicos = [topico.strip() for topico in topicos_input.split(',')]

        # Verificar se a lista de tópicos não está vazia
        if topicos and all(topico for topico in topicos):
            inputs_array = [{'tópico': topico} for topico in topicos]
            results = resumo_crew.kickoff_for_each(inputs=inputs_array)
            print("Pesquisa concluída. Verifique os resultados para os tópicos informados.")
            return results
        else:
            print("Entrada inválida. Por favor, informe pelo menos um tópico.")

def criar_estrategias():
    while True:
        notas = input("Informe suas últimas notas obtidas em linguagens, humanas, natureza, matemática e redação\n(utlize '.' para as notas e as separe por vírgula): ").split(',')
        notas = [t.strip() for t in notas]  # Remover espaços em branco

        # Verificar se a quantidade de notas é a esperada
        if len(notas) == 5 and all(nota.replace('.', '', 1).isdigit() for nota in notas):
            inputs = {
                'linguagens': notas[0],
                'humanas': notas[1],
                'natureza': notas[2],
                'matemática': notas[3],
                'redação': notas[4]
            }
            results = estrategias_crew.kickoff(inputs=inputs)
            print("Plano de estratégias criado com sucesso!")
            return results
        else:
            print("Entrada inválida. Por favor, insira 5 notas válidas separadas por vírgula.")


## Menu de interação com o usuário

In [58]:

# Menu interativo
def menu():
    while True:
        print("\nMenu:")
        print("1. Criar plano de estudos")
        print("2. Pesquisar tópico(s)")
        print("3. Elaborar estratégias")
        print("4. Sair")
        choice = input("Escolha uma opção (1-5): ")

        if choice == '1':
            criar_plano_de_estudos()
        elif choice == '2':
            pesquisar_topico()
        elif choice == '3':
            criar_estrategias()
        elif choice == '4':
            print("Saindo do programa...")
            break
        else:
            print("Opção inválida! Tente novamente.")

menu()


Menu:
1. Criar plano de estudos
2. Pesquisar tópico(s)
3. Elaborar estratégias
4. Sair
Escolha uma opção (1-5): 1
Informe suas últimas notas obtidas em linguagens, humanas, natureza, matemática e redação
(utlize '.' para as notas e as separe por vírgula): 753,804,732,896,400
# Agent: Diagnosticar o candidato
## Task: Realizar uma análise detalhada do desempenho do candidato com base nas notas de linguagens 753, humanas 804, natureza 732, matemática 896 e redação 400 do candidato,comparando-as com as médias e distribuições de notas das edições anteriores do ENEM extraídas dos arquivos PDF.


# Agent: Diagnosticar o candidato
## Final Answer: 
## Relatório de Diagnóstico:

### Notas do ENEM do Candidato:
- Linguagens: 753
- Humanas: 804
- Natureza: 732
- Matemática: 896
- Redação: 400

### Análise:
- As notas do candidato variam de 400 a 896, abrangendo diferentes níveis de desempenho em cada área do conhecimento.
- Destaca-se a nota mais alta em Matemática (896), demonstrando um forte 

# Agent: Pesquisador de Tópicos
## Task: Pesquisar informações detalhadas sobre o geometria analítica nos arquivos PDF das provas do ENEM (2011-2023),incluindo uma análise das questões relacionadas.


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.87it/s]




# Agent: Pesquisador de Tópicos
## Thought: I have searched for the keyword "geometria" in the ENEM 2018 and 2019 PDF files. Now, I need to extract relevant information about geometry topics in the exams to provide a detailed analysis.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"geometria\", \"pdf\": \"ENEM_2018.pdf\"}"
## Tool Output: 
Relevant Content:



Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.07it/s]


...

# Agent: Pesquisador de Tópicos
## Task: Pesquisar informações detalhadas sobre o física clássica nos arquivos PDF das provas do ENEM (2011-2023),incluindo uma análise das questões relacionadas.


Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  5.46it/s]




# Agent: Pesquisador de Tópicos
## Thought: I should start by searching for detailed information on classical physics in the ENEM PDF files. I will search for relevant content related to classical physics to provide a comprehensive summary on the topic.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"f\\u00edsica cl\\u00e1ssica\", \"pdf\": \"ENEM_PDF_files\"}"
## Tool Output: 
Relevant Content:



# Agent: Pesquisador de Tópicos
## Final Answer: 
# Física Clássica no ENEM

## Visão Geral do Tópico
A análise dos arquivos PDF do ENEM revelou que o tópico de Física Clássica foi abordado nas provas com questões que envolviam conceitos fundamentais da mecânica newtoniana, termodinâmica, óptica geométrica, eletromagnetismo clássico, entre outros. Os candidatos devem estar familiarizados com os princípios básicos da física clássica e sua aplicação em situações práticas.

## Questões Mais Frequentes
- Interpretação de gráficos de movimento retilíneo uniforme e uniformeme

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  3.71it/s]




# Agent: Diagnosticar o candidato
## Thought: I can now create the detailed analysis report based on the candidate's scores and compare them with the historical ENEM data extracted from the PDF.
## Using tool: Search a PDF's content
## Tool Input: 
"{\"query\": \"ENEM notas m\\u00ednimas e m\\u00e1ximas\", \"pdf\": \"file-sample_100kB.pdf\"}"
## Tool Output: 
Relevant Content:



# Agent: Diagnosticar o candidato
## Final Answer: 



# Agent: Estrategista de estudos e resolução de provas do ENEM
## Task: Propor estratégias de estudo e resolução de prova personalizadas com base no diagnóstico de desempenho do candidato.


# Agent: Estrategista de estudos e resolução de provas do ENEM
## Final Answer: 
# Plano Personalizado de Estudo e Resolução de Provas do ENEM

## Análise de Desempenho do Candidato:

O candidato apresentou as seguintes notas no ENEM: 689, 735, 697, 341 e 920. Após análise das notas, identificamos áreas de excelência e de oportunidade para melhoria.

- **Pontos Forte